In [1]:
import torch
import matplotlib.pyplot as plt
import scipy.io
from tqdm import tqdm
from FNO import FNO1d, get_dataloader

# -- Load and preprocess data --
data = scipy.io.loadmat('data/burgers_data_R10.mat')
a, u = data['a'], data['u']
a = torch.from_numpy(a).unsqueeze(1).float()
u = torch.from_numpy(u).unsqueeze(1).float()

In [2]:

# add spatial mesh as extra channel
mesh = torch.linspace(0, 2*torch.pi, u.size(-1))
mesh_b = mesh.view(1,1,-1).repeat(u.size(0),1,1)
a_with_mesh = torch.cat((a, mesh_b), dim=1)

# split train/test
train_input  = a_with_mesh[:1000]
train_label  = u[:1000]
test_input   = a_with_mesh[1000:1200]
test_label   = u[1000:1200]

In [3]:
# -- Model, optimizer, loader setup --
device      = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_c, out_c = 2, 1
modes, width= 16, 64
fno         = FNO1d(in_c, out_c, modes, width, torch.relu).to(device)

batch_size  = 100
train_loader= get_dataloader(train_input, train_label, batch_size)
test_loader = get_dataloader(test_input, test_label, batch_size, shuffle=False)

optimizer   = torch.optim.Adam(fno.parameters(), lr=3e-4)
epochs      = 10


/opt/anaconda3/envs/General/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# -- Training --
history = fno.train_model(train_loader, test_loader, optimizer, epochs, device)


  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
# -- Plot losses --
plt.figure()
plt.plot(history['train_loss'], label='train loss')
plt.plot(history['val_loss'],   label='val loss')
plt.yscale('log'); plt.legend()
plt.title('Training History')
plt.show()

# -- Plot a sample prediction --
pos = 3
x0  = test_input[pos:pos+1].to(device)
y0  = test_label[pos:pos+1].to(device)
pred= fno(x0).cpu().detach().numpy()[0,0,:]

plt.figure()
plt.plot(x0.cpu().numpy()[0,0,::32], label='initial')
plt.plot(y0.cpu().numpy()[0,0,::32], label='truth')
plt.plot(pred[::32],               label='prediction')
plt.legend(); plt.title(f'Sample #{pos}')
plt.show()